<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/RAG_for_Hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU transformers
!pip install -qU sentence_transformers
!pip install -qU huggingface_hub
!pip install -qU tiktoken
!pip install -qU datasets
!pip install -qU dspy-ai
!pip install -qU accelerate
!pip install -qU dspy-ai[chromadb]

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dspy-ai 2.4.5 requires datasets<3.0.0,~=2.14.6, but you have datasets 2.18.0 which is incompatible.


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import dspy
from dspy.datasets import DataLoader
data_loader = DataLoader()

In [ ]:
history_data = data_loader.from_huggingface('kaifahmad/indian-history-hindi-QA-3.4k')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
history_data = list(history_data.values())

**Model Loading:**

In [ ]:
gemma = dspy.HFModel(model = 'google/gemma-7b')

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

**Embedding:**

In [ ]:
import chromadb.utils.embedding_functions as embedding_functions
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key="YOUR_API_KEY",
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

**Retriver:**

In [ ]:
from dspy.retrieve.chromadb_rm import ChromadbRM
retriever_model = ChromadbRM(
    'your_collection_name',
    '/path/to/your/db',
    embedding_function=embedding_function,
    k=5
)
results = retriever_model("Explore the significance of quantum computing", k=5)
for result in results:
    print("Document:", result.long_text, "\n")

**Building Signature:**

In [ ]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers in Hindi."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="often relevant to question if not so display an aplology message")

In [ ]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [ ]:
from dspy.teleprompt import BootstrapFewShot
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

teleprompter = BootstrapFewShot(metric=validate_context_and_answer)
compiled_rag = teleprompter.compile(RAG(), trainset=history_data)

100%|██████████| 1/1 [00:00<00:00, 42.17it/s]

Failed to run or to evaluate example [Example({'Question': 'प्राचीन मानवों के भारतीय उपमहाद्वीप में प्रवास के प्रमुख सम्भावित मार्ग क्या थे?', 'Answer': 'प्राचीन मानवों का भारतीय उपमहाद्वीप में प्रवास संभवतः अरब सागर और बंगाल की खाड़ी के समुद्रतटीय मार्गों के साथ ही उत्तर-पश्चिमी प्रवासी द्वार के माध्यम से भी हो सकता था।'}) (input_keys=set()), Example({'Question': 'प्राचीन मानवों को भारतीय उपमहाद्वीप में प्रवास करने के लिए कौन-कौन से कारक प्रेरित कर सकते थे?', 'Answer': 'प्राचीन मानवों का भारतीय उपमहाद्वीप में प्रवास विभिन्न कारकों से प्रेरित हो सकता था, जैसे कि पर्यावरणीय परिवर्तन, नए संसाधनों की खोज, जनसंख्या की दबाव, और सांस्कृतिक आदान-प्रदान।'}) (input_keys=set()), Example({'Question': ' प्राचीन मानवों के भारतीय उपमहाद्वीप में प्रवास के सिद्धांतों में समुद्री मार्गों का क्या महत्व है?', 'Answer': 'समुद्री मार्गों का महत्वयान प्राचीन मानवों के भारतीय उपमहाद्वीप में प्रवास में हो सकता है, जिसमें अरब सागर और बंगाल की खाड़ी के समुद्रतटीय क्षेत्रों के साथ मार्गानुसार प्रवास हो सकता है।'

In [ ]:
# Ask any question you like to this simple RAG program.
my_question = "प्रारंभिक मानवों ने भारतीय उपमहाद्वीप के स्थलीय वन्य जीवन के साथ किस प्रकार के संबंध बनाए?"

# Get the prediction. This contains `pred.context` and `pred.answer`.
pred = compiled_rag(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

AssertionError: No RM is loaded.